In [11]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [12]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 4
    STEPS_PER_EPOCH = 100
    # not going to use these
    N_DISTORTIONS = 0
    MAX_DISTORTION = 0.3
    MIN_DISTORTION = -0.1
    NUM_WORKERS = 16
    PIN_MEMORY = True
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    # only flips
    IMAGE_AUGMENT = True
    DATA_ORDER = "ins"
    MEAN_PIXEL = np.array([0.485, 0.456, 0.406],dtype=np.float32).reshape(1,1,-1)
    STD_PIXEL = np.array([0.229, 0.224, 0.225],dtype=np.float32).reshape(1,1,-1)
    MAX_GT_INSTANCES = 100
    DETECTION_MAX_INSTANCES = 100
    DETECTION_MIN_CONFIDENCE = 0.7
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    DETECTION_NMS_THRESHOLD = 0.3
    LEARNING_RATE = 0.05
    LEARNING_MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0001
    WIDTH = 224
    HEIGHT = 224
    MASK_SHAPE = (64,64)
    GRID_WIDTH = 16
    GRID_HEIGHT = 16
    CLUE_SHAPE = (20,20)
    GRID_SHAPE = (GRID_WIDTH, GRID_HEIGHT)
    GRID_RESOLUTION = (1, 1)
    IS_PADDED = True
    MASK_THRESOLD = 0.7
    CROP_SIZE = 224
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)
        self.MAX_BATCH_SIZE = self.BATCH_SIZE*32

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [13]:
val_image_dir = "/media/data/nishanth/aravind/val2017/"
train_image_dir = "/media/data/nishanth/aravind/val2017/"
config = ProposalConfig()
model_dir = "./models/"
train_pickle = "/home/aravind/re/data/val_cwid.pickle"
val_pickle = "/home/aravind/re/data/val_cwid.pickle"

In [14]:
with open(train_pickle,"rb") as train_ann:
    train_cwid = pickle.load(train_ann)
with open(val_pickle,"rb") as val_ann:
    val_cwid = pickle.load(val_ann)

In [15]:
train_loader = model_lib.get_loader(train_cwid,config,train_image_dir)
val_loader = model_lib.get_loader(val_cwid,config,val_image_dir)


In [6]:
coco_dataset = model_lib.CocoDataset(train_cwid, config, train_image_dir)
coco_dataset.visualize_data()
# l = np.array(coco_dataset.cw_num_instances)
# print(l)
# print(np.sum(l[1:]/80),np.sum(l[2:]/79))
# l = np.log2(l)
# l = l**3.5
# print(l)

[  267 11004   316  1932   371   143   285   190   415   430   637   101
    75    60   413   440   202   218   273   361   380   255    71   268
   232   371   413   540   254   303   115   241    69   263   336   146
   148   179   269   225  1025   343   899   215   326   253   626   379
   239   177   287   316   371   127   285   338   316  1791   261   343
   163   697   179   288   231   106   283   153   262    55   143     9
   225   126  1161   267   277    36   191    11    57]
person


KeyboardInterrupt: 

In [ ]:
train_data =enumerate(train_loader)
counts = np.zeros(81)
from PIL import Image
for i in range(1000):
    start = time.time()
    idx,data = next(train_data)
    for j in data[0].numpy():
        z = ((np.moveaxis(j,0,-1))*config.STD_PIXEL+config.MEAN_PIXEL)*255
        Image.fromarray(z.astype(np.uint8),"RGB").show()
    input()
    print(np.argmax(np.array(data[-1]),-1))
    counts[np.argmax(np.array(data[-1]),-1)] += 1
    print(list(zip(range(81),counts)))
    end = time.time()
    print(end-start)

[  267 11004   316  1932   371   143   285   190   415   430   637   101
    75    60   413   440   202   218   273   361   380   255    71   268
   232   371   413   540   254   303   115   241    69   263   336   146
   148   179   269   225  1025   343   899   215   326   253   626   379
   239   177   287   316   371   127   285   338   316  1791   261   343
   163   697   179   288   231   106   283   153   262    55   143     9
   225   126  1161   267   277    36   191    11    57][  267 11004   316  1932   371   143   285   190   415   430   637   101
    75    60   413   440   202   218   273   361   380   255    71   268
   232   371   413   540   254   303   115   241    69   263   336   146
   148   179   269   225  1025   343   899   215   326   253   626   379
   239   177   287   316   371   127   285   338   316  1791   261   343
   163   697   179   288   231   106   283   153   262    55   143     9
   225   126  1161   267   277    36   191    11    57]
[  267 11004 


[27  1 44  1]
[(0, 0.0), (1, 1.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 1.0), (25, 0.0), (26, 0.0), (27, 1.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 1.0), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.0), (42, 1.0), (43, 0.0), (44, 1.0), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.0), (60, 0.0), (61, 0.0), (62, 0.0), (63, 0.0), (64, 0.0), (65, 0.0), (66, 0.0), (67, 0.0), (68, 0.0), (69, 0.0), (70, 0.0), (71, 0.0), (72, 0.0), (73, 0.0), (74, 0.0), (75, 1.0), (76, 0.0), (77, 0.0), (78, 0.0), (79, 0.0), (80, 0.0)]
14.434370040893555

[59  1  1  3]
[(0, 0.0), (1, 2.0), (2, 0.0), (3, 1.0), (4, 0.0), (5, 0.0), (6, 0.0)